<a href="https://colab.research.google.com/github/huiesoo/esaa/blob/main/24-1/0325.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 9 비지도학습

## 9.1 군집



*   군집

    비슷한 샘플을 구별해 하나의 클러스터 또는 비숫한 샘플의 그룹으로 할당하는 작업

*   고객 분류(-> 추천 시스템), 데이터 분석, 차원 축소 기법, 이상치 탐지, 준지도 학습, 검색 엔진, 이미지 분할



### 9.1.1 k-평균

In [1]:
from sklearn.datasets import make_blobs

In [2]:
import numpy as np
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

In [3]:
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [4]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_transform(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(




*   각 샘플은 k개의 클러스터 중 하나에 할당
*   군집에서 각 샘플의 레이블은 알고리즘이 샘플에 할당한 클러스터의 인덱스(labels_)



In [5]:
y_pred

array([[0.11146795, 3.04611916, 1.54944305, 1.45402521, 0.46779778],
       [0.51431557, 3.11541584, 1.48612753, 0.99002955, 0.07122059],
       [3.76340605, 1.32016676, 2.67154781, 4.09069201, 3.81713488],
       ...,
       [1.42865797, 3.04886464, 1.40795651, 0.06769209, 0.92830156],
       [3.23385668, 0.14895409, 1.71125   , 3.05913478, 3.10300136],
       [0.67518173, 2.8625311 , 1.21678483, 0.85434589, 0.22700281]])

In [6]:
y_pred is kmeans.labels_

False

In [7]:
kmeans.cluster_centers_

array([[-2.80037642,  1.30082566],
       [ 0.20876306,  2.25551336],
       [-1.46679593,  2.28585348],
       [-2.79290307,  2.79641063],
       [-2.80389616,  1.80117999]])

In [8]:
X_new = np.array([[0,2], [3,2], [-3,3], [-3,2.5]])
kmeans.predict(X_new)

array([1, 1, 3, 3], dtype=int32)



*   하드 군집

    *   샘플을 하나의 클러스터에 할당

*   소프트 군집

    *   클러스터마다 샘플에 점수를 부여



In [9]:
kmeans.transform(X_new)

array([[2.88633901, 0.32995317, 1.49439034, 2.9042344 , 2.81093633],
       [5.84236351, 2.80290755, 4.4759332 , 5.84739223, 5.80730058],
       [1.71086031, 3.29399768, 1.69136631, 0.29040966, 1.21475352],
       [1.21567622, 3.21806371, 1.54808703, 0.36159148, 0.72581411]])



*   k-평균 알고리즘
    *   데이터셋에 있는 모든 샘플에 가장 가까운 센트로이드의 클러스터 할당 / 모든 샘플의 레이블이 주어진다면 각 클러스터에 속한 샘플의 평균을 계산하여 모든 센트로이드를 쉽게 구함 / 레이블이나 센트로이드가 주어지지 않으면 처음에 센트로이드를 랜덤하게 선정 -> 샘플에 레이블 할당, 센트로이드 업데이트 -> 센트로이드에 변화가 없을 때까지 계속 -> 제한된 횟수 안에 수렴 보장
    *   가장 빠른 군집 알고리즘 중 하나



In [12]:
good_init = np.array([[-3,3], [-3,2], [-3,1], [-1,2], [0,2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1)
kmeans.fit(X)

KMeans(init=array([[-3,  3],
       [-3,  2],
       [-3,  1],
       [-1,  2],
       [ 0,  2]]),
       n_clusters=5, n_init=1)



*   이너셔

    성능 지표. 각 샘플과 가장 가까운 센트로이드 사이의 평균 제곱 거리



In [13]:
kmeans.inertia_

211.5985372581684

In [14]:
kmeans.score(X)

-211.59853725816836

In [15]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


MiniBatchKMeans(n_clusters=5)



*   미니배치 k-평균 알고리즘이 k-평균 알고리즘보다 훨씬 빠르지만 이너셔는 일반적으로 조금 더 나쁨(특히 클러스터의 개수가 증가할 때)





*   실루엣 점수
    *   모든 샘플에 대한 실루엣 계수의 평균
        *   샘플의 실루엣 계수 = (b-a)/max(a,b)
            *   a: 동일한 클러스터에 있는 다른 샘플까지 평균 거리(클러스터 내부의 평균 거리)
            *   b: 가장 가까운 클러스터까지 평균 거리(가장 가까운 클러스터의 샘플까지 평균 거리)
            *   -1 ~ +1
                
                +1에 까가우면 자신의 클러스터 안에 잘 속해 있고 다른 클러스터와는 멀리 떨어져 있다는 뜻




In [16]:
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

0.655517642572828

### 9.1.2 k-평균의 한계



*   타원형 클러스터에서는 가우시안 혼합 모델이 잘 작동



### 9.1.3 군집을 사용한 이미지 분할



*   이미지 분할

    이미지를 세그먼트 여러 개로 분할하는 작업

*   시맨틱 분할

    동일한 종류의 물체에 속한 모든 픽셀은 같은 세그먼트에 할당



In [19]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "unsupervised_learning"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

In [20]:
import urllib.request

homl3_root = "https://github.com/ageron/handson-ml3/raw/main/"
filename = "ladybug.png"
filepath = IMAGES_PATH / filename
if not filepath.is_file():
    print("Downloading", filename)
    url = f"{homl3_root}/images/unsupervised_learning/{filename}"
    urllib.request.urlretrieve(url, filepath)

In [21]:
import PIL

image = np.asarray(PIL.Image.open(filepath))
image.shape

(533, 800, 3)

In [23]:
X = image.reshape(-1,3)
kmeans = KMeans(n_clusters=8).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


### 9.1.4 군집을 이용한 전처리

In [24]:
from sklearn.datasets import load_digits

X_digits, y_digits = load_digits(return_X_y=True)

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

In [26]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:
log_reg.score(X_test, y_test)

0.9666666666666667

In [28]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50)),
    ("log_reg", LogisticRegression())
])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                ('log_reg', LogisticRegression())])

In [29]:
pipeline.score(X_test, y_test)

0.9711111111111111

In [30]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2,100))
grid_dlf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_dlf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Fitting 3 folds for each of 98 candidates, totalling 294 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.6s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

[CV] END ..............................kmeans__n_clusters=99; total time=   1.1s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ..............................kmeans__n_clusters=99; total time=   1.2s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                                       ('log_reg', LogisticRegression())]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [31]:
grid_dlf.best_params_

{'kmeans__n_clusters': 56}

In [32]:
grid_dlf.score(X_test, y_test)

0.9622222222222222

### 9.1.5 군집을 이용한 준지도 학습

In [33]:
n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [34]:
log_reg.score(X_test, y_test)

0.7622222222222222



*   대표 이미지

    각 클러스터에서 센트로이드에 가장 가까운 이미지



In [36]:
k = 50
kmeans = KMeans(n_clusters=k)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [37]:
y_representative_digits = np.array([
    0, 1, 3, 2, 7, 6, 4, 6, 9, 5,
    1, 2, 9, 5, 2, 7, 8, 1, 8, 6,
    3, 1, 5, 4, 5, 4, 0, 3, 2, 6,
    1, 7, 7, 9, 1, 8, 6, 5, 4, 8,
    5, 3, 3, 6, 7, 9, 7, 8, 4, 9])

In [38]:
log_reg = LogisticRegression()
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.12222222222222222



*   레이블 전파

    레이블을 동일한 클러스터에 있는 모든 샘플로 전파



In [39]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)
for i in range(k):
  y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

In [40]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.08444444444444445

In [42]:
percentile_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
  in_cluster = (kmeans.labels_==i)
  cluster_dist = X_cluster_dist[in_cluster]
  cutoff_distance = np.percentile(cluster_dist, percentile_closest)
  above_cutoff = (X_cluster_dist > cutoff_distance)
  X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [43]:
log_reg = LogisticRegression()
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.09333333333333334

In [44]:
np.mean(y_train_partially_propagated == y_train[partially_propagated])

0.08333333333333333



*   능동학습

    *   전문가가 학습 알고리즘과 상호작용하여 알고리즘이 요청할 때 특정 샘플의 레이블을 제공

    *   불확실성 샘플링



### 9.1.6 DBSCAN



*   밀집된 연속적 지역을 클러스터로 정의

*   작동 방식
    *   알고리즘이 각 샘플에거 작은 거리인 입실론 내에 샘플이 몇개 놓여 있는지 셈. 이 지역을 샘플의 입실론-이웃이라고 부름.
    *   입실론-이웃 내에 적어도 min_samples개 샘플이 있다면 이를 핵심 샘플로 간주. 즉 핵심 샘플은 밀집된 지역에 있는 샘플.
    *   핵심 샘플의 이웃에 있는 모든 샘플은 동일한 클러스터에 속함. 이웃에는 다른 핵심 샘플이 포함될 수 있음. 따라서 핵심 샘플의 이웃의 이웃은 계속해서 하나의 클러스터를 형성.
    *   핵심 샘플이 아니고 이웃도 아닌 샘플은 이상치로 판단



In [45]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [46]:
dbscan.labels_

array([ 0, -1,  1,  2,  2,  2,  3,  2,  3,  4,  5,  2,  3, -1,  1,  6,  2,
        2,  2,  5,  1,  5,  2,  0,  2,  3,  2,  3,  6,  3,  6,  2,  3,  3,
        6,  2,  3,  2,  6,  2, -1,  3,  3,  6,  2,  2,  1,  6,  2,  2,  2,
        2,  2,  2, -1,  3,  3,  2,  2,  6,  2,  2,  3,  3, -1,  2,  2,  6,
        2,  3, -1,  2, -1, -1,  2,  2,  1,  3,  2,  2,  0,  2,  0,  6,  6,
        2,  5,  2,  2,  3,  1, -1,  5,  2,  1,  1,  2,  3,  4,  2,  5,  3,
        1,  0,  5,  3,  6, -1, -1,  3,  3,  3,  2,  5,  1,  3,  2,  3,  1,
        3, -1,  1,  3,  6,  6,  5,  3,  2,  3,  3, -1,  0,  3,  3,  3,  1,
        2,  3,  0,  0,  2,  6,  2,  2,  3,  2,  7,  3,  3,  6, -1,  3,  1,
        1,  0,  1,  2,  1,  3,  3,  2,  2,  2,  2,  2,  6,  2,  3,  0,  3,
        2,  2,  2,  2,  3, -1,  2,  3,  2,  7,  6,  4,  2,  5,  2,  2,  2,
        2,  2,  3, -1,  5,  3,  0,  6,  2,  2,  2,  2,  2,  3,  3,  3,  2,
        0, -1,  1,  2,  5,  3,  6,  3,  3,  2,  3,  3,  5,  6,  3,  1,  6,
        6,  1,  3,  2,  6

In [49]:
len(dbscan.core_sample_indices_)

813

In [50]:
dbscan.core_sample_indices_

array([  0,   2,   3,   4,   7,   8,   9,  10,  11,  12,  14,  16,  17,
        18,  19,  20,  21,  22,  24,  26,  27,  28,  30,  31,  32,  33,
        34,  35,  36,  37,  38,  41,  42,  43,  44,  45,  46,  47,  48,
        49,  50,  51,  52,  53,  56,  57,  59,  61,  63,  65,  68,  71,
        74,  76,  77,  78,  79,  81,  82,  83,  84,  85,  86,  87,  88,
        89,  90,  92,  93,  94,  95,  96,  97,  98,  99, 101, 102, 103,
       105, 106, 109, 110, 111, 112, 114, 115, 116, 117, 118, 119, 121,
       122, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137,
       138, 139, 140, 141, 142, 143, 144, 145, 148, 149, 151, 152, 153,
       154, 155, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 167,
       169, 170, 171, 172, 173, 174, 176, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 192, 193, 195, 196, 197, 199, 200,
       201, 202, 203, 204, 206, 207, 208, 209, 210, 211, 212, 213, 215,
       216, 217, 218, 219, 220, 222, 223, 225, 227, 228, 229, 23

In [51]:
dbscan.components_

array([[ 0.06097644,  0.34206486],
       [ 2.02716692,  0.31372657],
       [-0.8844006 ,  0.49175459],
       ...,
       [ 1.0258932 , -0.49468739],
       [-0.11433256,  0.99472134],
       [ 0.91617446, -0.46734794]])

In [53]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

KNeighborsClassifier(n_neighbors=50)

In [54]:
X_new = np.array([[-0.5,0], [0,0.5], [1,-0.1], [2,1]])

In [55]:
knn.predict(X_new)

array([2, 0, 3, 1])

In [56]:
knn.predict_proba(X_new)

array([[0.04, 0.  , 0.88, 0.  , 0.  , 0.  , 0.08, 0.  ],
       [0.68, 0.  , 0.  , 0.  , 0.  , 0.  , 0.32, 0.  ],
       [0.  , 0.  , 0.  , 0.38, 0.38, 0.24, 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [57]:
y_dist, y_pred_idx = knn.kneighbors(X_new, n_neighbors=1)
y_pred = dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist>0.2] = -1
y_pred.ravel()

array([-1,  0,  4, -1])

### 9.1.7 다른 군집 알고리즘



*   병합 군집
*   BIRCH
*   평균-이동
*   유사도 전파
*   스펙트럼 군집



## 9.2 가우시안 혼합

In [58]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [59]:
gm.weights_

array([0.20098051, 0.2072646 , 0.59175489])

In [60]:
gm.means_

array([[ 1.75183683, -0.0565036 ],
       [-0.73820966,  0.57021766],
       [ 0.50599804,  0.24268023]])

In [62]:
gm.covariances_

array([[[ 0.05001398,  0.05834587],
        [ 0.05834587,  0.08319924]],

       [[ 0.05512907,  0.0636839 ],
        [ 0.0636839 ,  0.08919867]],

       [[ 0.16909668, -0.10152647],
        [-0.10152647,  0.28753906]]])

In [63]:
gm.converged_

True

In [64]:
gm.n_iter_

15

In [65]:
gm.predict(X)

array([2, 2, 0, 1, 1, 2, 2, 2, 0, 2, 2, 1, 0, 2, 0, 2, 1, 2, 1, 2, 0, 2,
       2, 2, 2, 0, 1, 2, 2, 0, 2, 1, 0, 2, 2, 1, 0, 1, 2, 1, 2, 0, 0, 2,
       2, 1, 0, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 0, 1, 1,
       1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2,
       1, 0, 0, 0, 2, 2, 0, 0, 1, 0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 1, 0,
       2, 0, 1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2,
       2, 0, 0, 0, 1, 0, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 0, 2, 1, 2, 0, 0,
       2, 0, 2, 0, 2, 0, 1, 1, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 1, 2, 2, 2,
       2, 0, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2,
       1, 2, 0, 2, 0, 1, 2, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0,
       2, 2, 0, 0, 1, 2, 2, 0, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 0, 1, 2, 0, 1,
       1, 2, 1, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 0, 1, 2, 2,
       2, 0, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2,

In [66]:
gm.predict_proba(X)

array([[3.80152482e-093, 5.24768857e-021, 1.00000000e+000],
       [9.76151648e-012, 4.63100470e-113, 1.00000000e+000],
       [9.99879694e-001, 6.25243566e-196, 1.20305858e-004],
       ...,
       [1.65464715e-004, 7.98228520e-146, 9.99834535e-001],
       [2.22213129e-164, 1.74294012e-002, 9.82570599e-001],
       [1.61643181e-007, 1.71227514e-131, 9.99999838e-001]])

In [67]:
X_new, y_new = gm.sample(6)

In [68]:
X_new

array([[ 2.11750425,  0.32098381],
       [-1.00810588,  0.27057795],
       [ 0.64277   ,  0.67695482],
       [ 0.62327074,  0.23494105],
       [ 0.45466463,  0.70690935],
       [ 0.47058768,  0.90654497]])

In [69]:
y_new

array([0, 1, 2, 2, 2, 2])

In [70]:
gm.score_samples(X)

array([-1.37930831e+00, -1.65351381e+00, -6.86441651e-01, -3.37648316e-01,
       -5.40182808e-02, -1.72984046e+00, -1.40384140e+00, -2.23587408e+00,
       -3.33881853e-01, -1.53305751e+00, -1.68773151e+00, -2.01107950e+00,
       -1.28039617e-01, -1.84323161e+00,  2.10976431e-02, -2.05456699e+00,
       -4.19865666e-01, -1.69863915e+00, -1.18105349e+00, -1.53649009e+00,
       -6.04013722e-01, -2.10915321e+00, -1.90533537e+00, -1.49434602e+00,
       -1.64700514e+00, -1.25127801e+00, -6.40974130e-01, -1.53043862e+00,
       -1.61743553e+00, -6.89273380e-01, -1.55444334e+00, -1.11256229e+00,
       -7.60721896e-01, -1.50858556e+00, -1.57113412e+00, -6.36644748e-01,
       -1.54057636e+00, -1.33341733e-01, -1.62266391e+00,  8.80188900e-02,
       -1.73437308e+00, -4.93791907e-01, -3.48465735e-01, -1.46412333e+00,
       -1.77517421e+00, -1.82888080e-01,  5.34548220e-02, -1.43807469e+00,
       -1.17667599e-01, -1.12175722e+00, -5.08850603e-01, -1.45004015e+00,
       -1.56160526e+00, -

In [74]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)
anomalies = X[densities < density_threshold]

In [75]:
gm.bic(X)

2747.6550706539115

In [76]:
gm.aic(X)

2664.223230911215

In [78]:
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(X)
np.round(bgm.weights_, 2)

/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:274: ConvergenceWarning: Initialization 10 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


array([0.13, 0.17, 0.13, 0.11, 0.11, 0.12, 0.11, 0.11, 0.  , 0.  ])